In [ ]:
# install trackml dependency from repository
!pip install git+https://github.com/LAL/trackml-library

In [ ]:
# download of zipped datasets
!mkdir datasets
!wget https://www.physi.uni-heidelberg.de/~dittmeier/pytorch/trackml_data/testset.zip
!wget https://www.physi.uni-heidelberg.de/~dittmeier/pytorch/trackml_data/valset.zip
!wget https://www.physi.uni-heidelberg.de/~dittmeier/pytorch/trackml_data/trainset.zip

In [ ]:
# and unzip them into our datasets directory
!unzip trainset.zip -d datasets
!unzip valset.zip -d datasets
!unzip testset.zip -d datasets

In [ ]:
# Imports
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import trackml.dataset
from trackml.utils import add_momentum_quantities
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# helper for GPU training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
# Data Visualization
event_path = "datasets/trainset/event000021000"

particles, hits, cells, truth = trackml.dataset.load_event(
    event_path, parts=["particles", "hits", "cells", "truth"]
)

print(truth.head())
print(truth["particle_id"].value_counts())

# Create histograms for particles
plt.figure(figsize=(12, 8))
for i, column in enumerate(particles.columns):
    plt.subplot(2, 5, i+1)
    plt.hist(particles[column], bins=50)
    plt.title(column)
plt.tight_layout()
plt.show()

# Create histograms for hits
plt.figure(figsize=(12, 8))
for i, column in enumerate(hits.columns):
    plt.subplot(2, 4, i+1)
    plt.hist(hits[column], bins=50)
    plt.title(column)
plt.tight_layout()
plt.show()

# Create histograms for cells
plt.figure(figsize=(12, 8))
for i, column in enumerate(cells.columns):
    plt.subplot(2, 3, i+1)
    plt.hist(cells[column], bins=50)
    plt.title(column)
plt.tight_layout()
plt.show()

# Create histograms for truth
plt.figure(figsize=(12, 8))
for i, column in enumerate(truth.columns):
    plt.subplot(2, 5, i+1)
    plt.hist(truth[column], bins=50)
    plt.title(column)
plt.tight_layout()
plt.show()


In [ ]:
# more visualisation, setup for a 3D plot
!pip install ipympl
from mpl_toolkits.mplot3d import Axes3D
from google.colab import output
output.enable_custom_widget_manager()
%matplotlib widget

In [ ]:
# Creating a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
hits_select = hits[hits.volume_id==8]
ax.scatter(hits_select.x, hits_select.y, hits_select.z, c='r', marker='.')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
plt.show()


In [ ]:
# Create a dataset, with a label for each hit, if hits have same particle_id set to 1, else -1
# and an option to apply a cut on pT on the dataset

class TrackMLDataset(Dataset):
    def __init__(self, event_path, nevents=10, cuts=None):
        self.event_path = event_path
        self.nevents = nevents
        self.event_ids = []
        self.particles = []
        self.hits = []
        self.cells = []
        self.truth = []
        self.labels = []

        for event_id, hits, cells, particles, truth in trackml.dataset.load_dataset(event_path, nevents=nevents):
            particles = add_momentum_quantities(particles)
            print(event_id)
            if cuts is not None:
                print(f"Applying cuts: {cuts}")
                particles = particles[particles.pt >= cuts["pt"]].reset_index(drop=True)
                truth = truth[truth.particle_id.isin(particles.particle_id)].reset_index(drop=True)
                hits  = hits[hits.hit_id.isin(truth.hit_id)].reset_index(drop=True)
                cells = cells[cells.hit_id.isin(hits.hit_id)].reset_index(drop=True)

            self.event_ids.append(event_id)
            self.particles.append(particles)
            self.hits.append(hits)
            self.cells.append(cells)
            self.truth.append(truth)

            labels = np.full((len(truth), len(truth)), -1, dtype=np.int8)    # this is not memory friendly!!!
            for particle_id, nhits in zip(particles.particle_id, particles.nhits):
                particle_hits = truth[truth.particle_id == particle_id].index
                for match in particle_hits:
                    for match2 in particle_hits:
                        labels[match, match2] = 1

            # think about metric learning approach; make use of volumes and layers

            self.labels.append(labels)

    def __len__(self):
        return len(self.event_ids)

    def __getitem__(self, idx):
        event_id = self.event_ids[idx]
        particles = self.particles[idx]
        hits = self.hits[idx]
        cells = self.cells[idx]
        truth = self.truth[idx]
        labels = self.labels[idx]
        return event_id, torch.tensor(particles.values, dtype=torch.float32), torch.tensor(hits.values, dtype=torch.float32), torch.tensor(cells.values, dtype=torch.float32), torch.tensor(truth.values, dtype=torch.float32), torch.from_numpy(labels).type(torch.int8)


In [ ]:
# Loading the dataset
print("Loading Trainset")
event_path = "datasets/trainset"
training_data = TrackMLDataset(event_path, nevents=80, cuts={"pt": 2})
print("Loading Valset")
event_path = "datasets/valset"
val_data = TrackMLDataset(event_path, nevents=10, cuts={"pt": 2})
print("Loading Testset")
event_path = "datasets/testset"
test_data = TrackMLDataset(event_path, nevents=10, cuts={"pt": 2})

In [1]:
# Setting up DataLoader
batch_size = 1
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

# Getting a first test event
event_id, particles, hits, cells, truth, labels = next(iter(train_dataloader))

print(f"event_id shape: {event_id.size()}; event_id data type: {event_id.dtype}")
print(f"particles shape: {particles.size()}; particles data type: {particles.dtype}")
print(f"hits shape: {hits.size()}; hits data type: {hits.dtype}")
print(f"cells shape: {cells.size()}; cells data type: {cells.dtype}")
print(f"truth shape: {truth.size()}; truth data type: {truth.dtype}")
print(f"labels shape: {labels.size()}; labels data type: {labels.dtype}")

print(f"event_id: {event_id.squeeze()}")
print(f"hit coordinates: {hits.squeeze()[:,1:4]}")    # x, y, z
print(f"labels: {labels.squeeze()}")



NameError: name 'DataLoader' is not defined

In [ ]:
# Model definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential( # Sequential container
            nn.Linear(3, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 8),
        )

    def forward(self, x):
        y = self.linear_relu_stack(x)
        return y

# instantiates the model and sends it to GPU
model = NeuralNetwork().to(device)

In [ ]:
learning_rate = 1e-3
batch_size = batch_size
epochs = 10
margin = 1.0

loss_fn = nn.HingeEmbeddingLoss(margin=margin)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# loops over our optimization code
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()

    for batch, [event_id, particles, hits, cells, truth, labels] in enumerate(dataloader):
        # Send data to GPU
        particles, hits, cells, truth, labels = to_device([particles, hits, cells, truth, labels], device)

        embedding = model(hits.squeeze()[:,1:4])
        distances = torch.cdist(embedding, embedding)

        loss = loss_fn(distances, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * batch_size + len(event_id)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        # Clear GPU memory
        del event_id, particles, hits, cells, truth, embedding, distances, labels, loss
        torch.cuda.empty_cache()



In [ ]:
# we want to do a knn from scikit
from sklearn.neighbors import NearestNeighbors

def evaluate(embeddings, labels, knn, radius):
  embeddings_cpu = embeddings.cpu().detach().numpy()

  neigh = NearestNeighbors(n_neighbors=knn, radius=radius)
  neigh.fit(embeddings_cpu)
  distances, indices = neigh.kneighbors(embeddings_cpu)
  #print(distances)
  #print(indices)


  positives = 0
  negatives = 0

  count_1 = torch.sum(labels == 1).item()-labels.size(dim=1)
  

  for ind, dist in zip(indices,distances):
    # we can just check with labels, if knn and label = 1 --> match, -1 --> bad
      valid_ind = ind[dist<radius]
      positives += torch.sum(labels.squeeze()[valid_ind[0],valid_ind[1:]] == 1).item()
      negatives += torch.sum(labels.squeeze()[valid_ind[0],valid_ind[1:]] == -1).item()
  #print(f"positives: {positives}; negatives: {negatives}; count_1: {count_1}")
  efficiency = positives/count_1
  purity = positives/(positives + negatives)
  return efficiency, purity

In [ ]:
# evaluate the model's performance against the test dataset
def test_loop(dataloader, model, loss_fn, device, knn, radius):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, efficiency, purity = 0, 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for event_id, particles, hits, cells, truth, labels  in dataloader:
            # Send data to GPU
            particles, hits, cells, truth, labels = to_device([particles, hits, cells, truth, labels], device)

            embedding = model(hits.squeeze()[:,1:4])
            distances = torch.cdist(embedding, embedding)

            test_loss += loss_fn(distances, labels).item()

            eff, pur = evaluate(embedding, labels, knn, radius)
            efficiency += eff
            purity += pur

            del event_id, particles, hits, cells, truth, embedding, distances, labels
            torch.cuda.empty_cache()

    test_loss /= num_batches
    efficiency /= num_batches
    purity /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} Efficiency: {efficiency:>8f} Purity: {purity:>8f} \n")
    return test_loss, efficiency, purity

In [ ]:
loss = []
eff  = []
pur  = []
knn = 21
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, device)
    l,e,p = test_loop(val_dataloader, model, loss_fn, device, knn, margin)
    loss.append(l)
    eff.append(e)
    pur.append(p)
print("Done!")
new_res = [loss, eff, pur]


In [ ]:
#for name, param in model.named_parameters():
#    print(f"Parameter name: {name}, Size: {param.size()}, Values: {param}")

# Assuming you have the `test_loss` variable containing the loss values for each epoch
epochs_r = range(1, len(loss) + 1)
for res, titles in zip (new_res, ["Loss", "Efficiency", "Purity"]):
    print(res)
    plt.figure(figsize=(6, 4))
    plt.plot(epochs_r, res, 'b', label=titles)
    plt.xlabel('Epoch')
    plt.ylabel(titles)
    plt.legend()
    plt.show()


In [ ]:
test_loop(test_dataloader, model, loss_fn, device, 50, margin)
test_loop(test_dataloader, model, loss_fn, device, 100, margin)